In [49]:
import pandas as pd
import requests
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
from datetime import datetime, timedelta
import random
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "deepseek-ai/deepseek-llm-7b-chat"
torch.cuda.set_device(3)  # Sets default to GPU 0
device=torch.device("cuda:3")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 3},             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.94s/it]


In [50]:
class fundamental_analyst:

    def __init__ (self, ticker):
        self.ticker=ticker
        self.get_financial_info()
        self.generate_prompt()
        
    def get_financial_info(self):
        ticker = yf.Ticker(self.ticker)
        self.income_stmt = ticker.financials
        self.balance_sheet = ticker.balance_sheet
        self.cash_flow = ticker.cashflow
        self.info = ticker.info
        self.year=self.income_stmt.columns[0]
        self.EPS=self.income_stmt[self.year]["Diluted EPS"]
        self.Net_Income=self.income_stmt[self.year]["Net Income"]
        self.Gross_Profit=self.income_stmt[self.year]["Gross Profit"]
        self.Revenue=self.income_stmt[self.year]["Total Revenue"]
        self.Total_Assets=self.balance_sheet[self.year]["Total Assets"]
        self.Total_Liabilities=self.balance_sheet[self.year]["Total Liabilities Net Minority Interest"]
        self.Shareholders_Equity=self.balance_sheet[self.year]["Stockholders Equity"]
        self.Free_Cash_Flow=self.cash_flow[self.year]["Free Cash Flow"]
        self.Investing_Cash_Flow=self.cash_flow[self.year]["Investing Cash Flow"]
        self.Financing_Cash_Flow=self.cash_flow[self.year]["Financing Cash Flow"]
        self.Operating_Cash_Flow=self.cash_flow[self.year]["Operating Cash Flow"]
        self.P_E=ticker.info['trailingPE']
        self.ROA=ticker.info['returnOnAssets']
        self.ROE=ticker.info['returnOnEquity']
    
    def generate_prompt(self):
        prompt = f"""
Pretend that you are a fundamental investment analyst. Analyze the financial performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
Do not add assumptions about company operations, reputation, or strategy.

Financials for {self.ticker}:

Income Statement:
Revenue: ${self.Revenue:,.0f}
Gross Profit: ${self.Gross_Profit:,.0f}
Net Income: ${self.Net_Income:,.0f}
EPS (Diluted): {self.EPS:.2f}

Balance Sheet:
Total Assets: ${self.Total_Assets:,.0f}
Total Liabilities: ${self.Total_Liabilities:,.0f}
Shareholders' Equity: ${self.Shareholders_Equity:,.0f}

Cash Flow:
Operating Cash Flow: ${self.Operating_Cash_Flow:,.0f}
Free Cash Flow: ${self.Free_Cash_Flow:,.0f}
Investing Cash Flow: ${self.Investing_Cash_Flow:,.0f}
Financing Cash Flow: ${self.Financing_Cash_Flow:,.0f}

Valuation and Ratios:
P/E Ratio: {self.P_E:.2f}
ROA: {self.ROA:.2%}
ROE: {self.ROE:.2%}

Based on this, what is your investment recommendation? Pick one action candidate."""
        self.prompt=prompt
        return self.prompt
        
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )
        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only

In [51]:
class technical_analyst:

    def __init__(self, ticker):
        self.ticker=ticker
        self.generate_df()
        self.generate_indicators()
        self.generate_technical_prompt()
        
    def generate_df(self):
        self.data=yf.download(self.ticker, period='1y')

    def compute_rsi(self, close, period=14):
        delta = close.diff()

        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def compute_obv(self, close, volume):
        direction = close.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
        obv = (volume * direction).fillna(0).cumsum()
        return obv
    
    def generate_indicators(self):
        temp=pd.DataFrame()
        temp["SMA_5"] = self.data['Close'][self.ticker].rolling(5).mean()
        temp["SMA_15"] = self.data['Close'][self.ticker].rolling(15).mean()
        temp["SMA_50"] = self.data['Close'][self.ticker].rolling(50).mean()

        temp['EMA_5'] = self.data['Close'][self.ticker].ewm(span=5).mean()
        temp['EMA_10'] = self.data['Close'][self.ticker].ewm(span=10).mean()
        temp['EMA_50'] = self.data['Close'][self.ticker].ewm(span=50).mean()
        temp["Date"] = self.data['Close'][self.ticker].index
        temp["RSI"]=self.compute_rsi(self.data['Close'][self.ticker])
        temp["OBV"]=self.compute_obv(self.data['Close'][self.ticker], self.data['Volume'][self.ticker])
        self.indicator_df=temp

    def generate_technical_prompt(self):
        latest = self.indicator_df.iloc[-1]
        prompt = f"""
ou are a technical investment analyst. Analyze the recent technical performance of {self.ticker} and give an investment recommendation.

Your task:
Choose one of the following recommendations: Strong Buy, Buy, Hold, Sell, or Short.
Justify your choice using 4–6 bullet points based only on the indicators below.
You MUST respond in natural human language. Do NOT include any code or formulas.

Technical Indicators for {self.ticker} (most recent data point):

SMA 5: {latest['SMA_5']:.2f}
SMA 15: {latest['SMA_15']:.2f}
SMA 50: {latest['SMA_50']:.2f}

EMA 5: {latest['EMA_5']:.2f}
EMA 10: {latest['EMA_10']:.2f}
EMA 50: {latest['EMA_50']:.2f}

RSI: {latest['RSI']:.2f}
OBV: {latest['OBV']:,.0f}

Based on this, what is your investment recommendation? Pick one action candidate.
"""
        self.prompt=prompt
        return self.prompt 
    
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only


In [56]:
class news_analyst:

    def __init__ (self,ticker, company):
        self.ticker=ticker
        self.company=company
        self.get_news_articles()
        self.generate_news_prompt()
        
    def get_news_articles(self):
        to_date = datetime.today().date()

        from_date = to_date - timedelta(days=7)

        from_str = from_date.strftime('%Y-%m-%d')
        to_str = to_date.strftime('%Y-%m-%d')

        symbol = self.ticker
        api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
        url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={from_str}&to={to_str}&token={api_key}"
        response = requests.get(url)
        news = response.json()
        filtered_news = [
            item for item in news
            if self.company in item['headline'].lower()
        ]
        self.selected_news = random.sample(filtered_news, k=min(10, len(filtered_news)))
    
    def generate_news_prompt(self):
        prompt = f"""
Pretend that you are a sentiment and headlines investment analyst. Analyze the recent technical performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.

Headlines:
"""
        for i, item in enumerate(self.selected_news):
            prompt+=item['headline']
            prompt+='\n'
        prompt+="Based on this, what is your investment recommendation? Pick one action candidate."
        self.prompt=prompt
        return self.prompt
    
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only

In [61]:
funda=fundamental_analyst('NVDA')
print(funda.prompt)
funda_analysis=funda.generate_response()
print("Response")
print(funda_analysis)

Device set to use cuda:3



Pretend that you are a fundamental investment analyst. Analyze the financial performance of NVDA and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
Do not add assumptions about company operations, reputation, or strategy.

Financials for NVDA:

Income Statement:
Revenue: $130,497,000,000
Gross Profit: $97,858,000,000
Net Income: $72,880,000,000
EPS (Diluted): 2.94

Balance Sheet:
Total Assets: $111,601,000,000
Total Liabilities: $32,274,000,000
Shareholders' Equity: $79,327,000,000

Cash Flow:
Operating Cash Flow: $64,089,000,000
Free Cash Flow: $60,853,000,000
Investing Cash Flow: $-20,421,000,000
Financing Cash Flow: $-42,359,000,000

Valuation and Ratios:
P/E Ratio: 54.91
ROA: 53.24%
ROE: 115.46%

Based on this, what is your investment recommendation? Pick one action candidate.
Response
As an AI language model, I ca

In [59]:
techni=technical_analyst('NVDA')
print(techni.prompt)
techni_analysis=techni.generate_response()
print("Response:")
print(techni_analysis)

/tmp/ipykernel_3781275/4036647504.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.data=yf.download(self.ticker, period='1y')
[*********************100%***********************]  1 of 1 completed
Device set to use cuda:3



ou are a technical investment analyst. Analyze the recent technical performance of NVDA and give an investment recommendation.

Your task:
Choose one of the following recommendations: Strong Buy, Buy, Hold, Sell, or Short.
Justify your choice using 4–6 bullet points based only on the indicators below.
You MUST respond in natural human language. Do NOT include any code or formulas.

Technical Indicators for NVDA (most recent data point):

SMA 5: 170.92
SMA 15: 165.83
SMA 50: 148.47

EMA 5: 169.91
EMA 10: 168.09
EMA 50: 150.09

RSI: 73.03
OBV: 3,116,559,600

Based on this, what is your investment recommendation? Pick one action candidate.

Response:
My recommendation is to "Buy" NVDA. The recent technical performance of NVDA has shown strong momentum, as indicated by the high RSI value of 73.03 and the positive OBV of 3,116,559,600. The SMA and EMA indicators also suggest a bullish trend, with the 5-day SMA at 170.92, the 15-day SMA at 165.83, the 50-day SMA at 148.47, the EMA 5 at 169.

In [60]:
news=news_analyst('NVDA','nvidia')
print(news.prompt)
news_analysis=news.generate_response()
print("Response:")
print(news_analysis)

Device set to use cuda:3



Pretend that you are a sentiment and headlines investment analyst. Analyze the recent technical performance of NVDA and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.

Headlines:
Why Is Wall Street Obsessed With AI (Artificial Intelligence) Leader Nvidia?
InnovizSMART Integrates NVIDIA Jetson Orin for Advanced Edge Computing
Stock Market Today: Dow Rises As Fed Officials Are Firm On Rates; Nvidia Scores Record High (Live Coverage)
Veteran fund manager who predicted Nvidia stock rally reboots forecast on China
Nvidia Rally Shows Signs of Overheating as Gains Blow Past 80%
Nvidia (NVDA) Stock Falls Amid Market Uptick: What Investors Need to Know
The Lucid-Uber Robotaxi Deal: How Nvidia Will Also Benefit
Nvidia, Alphabet, SMCI Rally With No Bubble Warning
IPO Stock Of The Week: Nvidia Partner Arm Ho